# CNN (MNIST)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

In [13]:
train_data = torchvision.datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = torchvision.transforms.ToTensor(), 
    download = True,            
) # train
test_data = torchvision.datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = torchvision.transforms.ToTensor()
) # validation

In [14]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: data
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [15]:
# mini batch
batch_size = 100
train_loader = torch.utils.data.DataLoader(
    dataset = train_data,
    batch_size = batch_size,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = test_data,
    batch_size = batch_size,
    shuffle = False
)

CNN: input ( 가로 * 세로 * 깊이 )  
깊이 = channels (for example, rgb => 3, 흑백 => 1)  


In [16]:
import torch.nn as nn
class cnn(nn.Module): 
    def __init__(self): 
        super(cnn, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3, padding=1) 
        self.conv2 = nn.Conv2d(12, 24, 3, padding=1)
        self.fc1 = nn.Linear(7*7*24, 128)
        self.fc2 = nn.Linear(128,10)

    def forward(self, x):
        # convoultion # input (28 * 28 * 1)
        x = self.conv1(x) # (28 * 28 * 12)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2) # (14 * 14 * 12)
        x = self.conv2(x) # (14 * 14 * 24)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2) # (7 * 7 * 24)
        
        # fully connected
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.softmax(x,dim=1)
        return x

In [17]:
model = cnn()

# loss and optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [18]:
class Print(nn.Module):
    def forward(self, x):
        print(x.size())
        return x

In [19]:
model

cnn(
  (conv1): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [20]:
# training
def train(model, criterion, optimizer, data):
    model.train() # training
    for X,y in data:
        y_pred = model(X)
        loss = criterion(y_pred,y)
        
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases

In [21]:
train(model, criterion, optimizer, train_loader)

In [24]:
def evaluation(model, criterion, optimizer, data):
    model.eval() #evaluation
    test_loss = 0
    accuracy = 0

    with torch.no_grad(): # no update
        for X, y in data:
            y_pred = model(X)
            test_loss += criterion(y_pred, y)
            
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy += corrects.sum().float() / float( y.size(0) )
    return test_loss, accuracy        

In [ ]:
nums_epoch = 10
for epoch in range(nums_epoch):
    train(model, criterion, optimizer, train_loader)
    
    if(epoch%2==0):
        E, accu = evaluation(model, criterion, optimizer, test_loader)
        print(epoch, "loss=", E.item(), "accu=", accu.item())

0 loss= 148.73348999023438 accu= 97.57998657226562
